# Mac OS를 위한 setting

In [30]:
# webdriver setting
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager

driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

# beautifulSoup, pandas setting
from bs4 import BeautifulSoup
import pandas as pd

# os, time setting
import os
import time
import copy
import re
import threading

### 크롤링하고자 하는 데이터
1. name: 글꼴 이름
2. author: 제작자
3. downloads: 다운로드 수
4. link: 폰트의 dafont 페이지 링크
5. theme: 폰트의 하위 테마
6. license: 폰트의 라이센스
7. supported_glyphs: 지원되는 문체 (accent, euro)
8. uploaded_year: dafont에 업로드된 년도

### 크롤링할 웹사이트
https://www.dafont.com/mtheme.php?id=1
dafont에서 'Fancy'라는 테마를 가진 글꼴들을 대상으로 크롤링한다.


In [33]:
font_data = {
    'name': None,
    'author': None,
    'downloads': None,
    'link': None,
    'theme': None,
    'license': None,
    'supported_glyphs': None,
    'uploaded_year': None
}

base_url = "https://www.dafont.com"
theme_url = f"{base_url}/mtheme.php?id=1"

fonts_info = []

page = 1
objective = 100

# 크롤링 시작
### 크롤링하는 순서와 방법론은 다음과 같다.
1. url 파라미터로 페이지를 결정한다. (f"{base_url}&page=n"에서 n을 조작함으로써 페이지를 옮길 수 있다.)
2. 이 페이지의 폰트 리스트는 하나의 html element로 묶여 있지 않다. "lv1left dfbg" class값을 가진 div태그들을 모아 그 안에 있는 a태그의 href값을 추출하고 합쳐 임시 폰트 url 배열을 만든다.
3. 임시 폰트 url 배열에 대해 순회하며 폰트 상세 페이지를 방문한다.
4. 데이터 수집을 끝마쳤으면 bs4 객체를 초기화하고 다음 페이지로 넘어간다.
5. 추출된 url이 없으면 끝낸다.

In [34]:
# 정보 추출을 위한 함수 정의
def extract_element(soup, selector, default="unknown", post_process=None):
    try:
        element = soup.select_one(selector)
        text = element.text.strip() if element else default
        return post_process(text) if post_process else text
    except Exception as e:
        print(f"An error occurred: {e}")
        return default

In [35]:
def extract_font_info(soup):
    font_info = copy.deepcopy(font_data)
    
    font_info['name'] = extract_element(soup, 'div.lv1left > strong')
    font_info['author'] = extract_element(soup, 'div.lv1left > a')
    font_info['downloads'] = extract_element(soup, 'div.lv2right > span', post_process=lambda x: int(re.search(r'(\d+)', x).group(1)) if re.search(r'(\d+)', x) else 0)
    font_info['link'] = soup.current_url if hasattr(soup, 'current_url') else "unknown"
    font_info['theme'] = extract_element(soup, 'div.lv1right > a:nth-of-type(2)')
    font_info['license'] = extract_element(soup, 'a.tdn.help.black')
    font_info['supported_glyphs'] = extract_element(soup, 'div.lv1left > span.contain', post_process=lambda x: ', '.join([elem.text.strip() for elem in soup.select('div.lv1left > span.contain')]) if soup.select('div.lv1left > span.contain') else "unknown")
    font_info['uploaded_year'] = extract_element(soup, 'div.dfsmall', post_process=lambda x: int(re.search(r'(\d{4})', x).group(1)) if re.search(r'(\d{4})', x) else "unknown")
    
    return font_info


In [36]:
while True:
    # 페이지 세팅 후 접속
    url = f"{theme_url}&page={page}"
    driver.get(url)
    
    time.sleep(0.5)
    
    # soup 받아오기
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # soup의 특정 class를 가진 요소들을 모조리 찾음
    elements_lv1left_dfbg = soup.find_all('div', class_="lv1left dfbg")
    
    # 없으면 크롤링 종료
    if not elements_lv1left_dfbg:
        break
    
    # 폰트 상세 페이지로 이동하는 url을 배열로 묶어냄 
    font_urls = [element.select_one('div.lv1left > a')['href'] for element in elements_lv1left_dfbg]
    

    for font_url in font_urls:
        # 추출한 url에 따라 페이지 방문
        driver.get(f"{base_url}/{font_url}")
        
        time.sleep(0.5)
        
        font_detail_soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # 폰트 정보 수집 후 저장
        font_info = extract_font_info(font_detail_soup)
        fonts_info.append(font_info)
        
        # 목표 데이터 수에 도달했다면 탈출
        if len(fonts_info) >= objective:
            break
    
    # 목표 데이터 수에 도달했다면 탈출
    if len(fonts_info) >= objective:
        break
        
    # 다음 페이지로 넘어감
    page += 1
        

#1 Saved font info: Starborn
#2 Saved font info: Varsity
#3 Saved font info: Melted Monster
#4 Saved font info: Cheese Burger
#5 Saved font info: Harry P
#6 Saved font info: Another Danger
#7 Saved font info: Super Milk
#8 Saved font info: Super Mario 256
#9 Saved font info: Super Plants
#10 Saved font info: Ghastly Panic
#11 Saved font info: Ruzack Mojok
#12 Saved font info: Krifon
#13 Saved font info: Doctor Glitch
#14 Saved font info: KG Red Hands
#15 Saved font info: Happy Dance
#16 Saved font info: Cheri
#17 Saved font info: Muloka Kerash
#18 Saved font info: American Captain
#19 Saved font info: Komika Axis
#20 Saved font info: Next Sunday
#21 Saved font info: Valoon
#22 Saved font info: Sunday Mango
#23 Saved font info: Horror Korpus
#24 Saved font info: BubbleGum
#25 Saved font info: Freshman
#26 Saved font info: Pokémon
#27 Saved font info: Badaboom BB
#28 Saved font info: Marola
#29 Saved font info: Adelia
#30 Saved font info: Who asks Satan
#31 Saved font info: Urban Jungle


TimeoutException: Message: Navigation timed out after 300000 ms
